In [41]:
import pandas as pd
from itertools import combinations as comb
import csv

def fetch_synergies_from_file(drug_ids, csv_file_path="drug_data.csv"):
    """Fetch and return synergy scores for analysis from a CSV file."""
    synergy_results = {}
    drug_data = pd.read_csv(csv_file_path)

    pairs = comb(drug_ids, 2)  # Generate all unique pairs from the given drug IDs

    for id1, id2 in pairs:
        # Find the combination in the dataframe
        row = drug_data[((drug_data['idDrugA'] == id1) & (drug_data['idDrugB'] == id2)) |
                ((drug_data['idDrugA'] == id2) & (drug_data['idDrugB'] == id1))]
        if not row.empty:
            scores = {
                'drugA':row.iloc[0]['drugNameA'],
                'drugB':row.iloc[0]['drugNameB'],
                'bliss': row.iloc[0]['bliss'],
                'zip': row.iloc[0]['zip'],
                'loewe': row.iloc[0]['loewe'],
                'hsa': row.iloc[0]['hsa']
            }
        else:
            scores = "Combination Data Not Found"
        
        synergy_results[(id1, id2)] = scores

    return synergy_results

import numpy as np

def count_responses(overall_synergy_responses):
    count_synergistic = 0
    count_additive = 0

    for response in overall_synergy_responses.values():
        if response == "Synergistic":
            count_synergistic += 1
        elif response == "Additive":
            count_additive += 1

    return count_synergistic, count_additive

def analyze_combination_synergy(synergy_data):
    """Analyze drug combination synergy using multiple models with refined HSA scoring."""
    overall_synergy_responses = {}
    reasoning = ""
    any_antagonism = False
    any_significant_synergy = False

    for pair, scores in synergy_data.items():
        drugA = scores['drugA']
        drugB = scores['drugB']
        loewe = scores['loewe']
        zip_score = scores['zip']
        hsa = scores['hsa']
        bliss = scores['bliss']

        # Use Bliss to further characterize interaction as synergistic or antagonistic
        if bliss > 0.1:
            bliss_response = "Synergistic"
        elif bliss < -0.1:
            bliss_response = "Antagonistic"
        else:
            bliss_response = "Additive"

        # Use Loewe score for primary synergy or antagonism analysis
        if loewe > 0.1:
            loewe_response = "Antagonistic"
        elif loewe < -0.1:
            loewe_response = "Synergistic"
        else:
            loewe_response = "Additive"

        # Adjust ZIP scores for extreme values
        if zip_score > 15:
            zip_response = "Highly Synergistic"
            any_significant_synergy = True
        elif zip_score < -15:
            zip_response = "Highly Antagonistic"
            any_antagonism = True
        elif zip_score > 0:
            zip_response = "Synergistic"
        elif zip_score < 0:
            zip_response = "Antagonistic"
            if zip_score < -1:
                any_antagonism = True
        else:
            zip_response = "Additive"

        # Adjust HSA score interpretation for interaction effects
        if hsa > 0.1:
            hsa_response = "Synergistic"
        elif hsa < -0.1:
            hsa_response = "Antagonistic"
        else:
            hsa_response = "Additive"

        # Combine responses to determine overall drug pair interaction
        key_responses = [bliss_response, loewe_response, hsa_response]
        if zip_response == "Highly Synergistic":
            overall_synergy_responses[pair] = "Synergistic"
            reasoning += f"{drugA} and {drugB} are highly synergistic. "
        elif zip_response == "Highly Antagonistic":
            overall_synergy_responses[pair] = "Antagonistic"
            reasoning += f"{drugA} and {drugB} are highly antagonistic. "
        elif key_responses.count('Antagonistic') > 1 or any_antagonism:
            overall_synergy_responses[pair] = "Antagonistic"
            reasoning += f"{drugA} and {drugB} are antagonistic. "
        elif key_responses.count('Synergistic') > 1:
            overall_synergy_responses[pair] = "Synergistic"
            reasoning += f"{drugA} and {drugB} are synergistic. "
        elif key_responses.count('Additive') > 1:
            overall_synergy_responses[pair] = "Additive"
            reasoning += f"{drugA} and {drugB} are additive. "
        else:
            overall_synergy_responses[pair] = loewe_response
            reasoning += f"{drugA} and {drugB} are {loewe_response}. "

    synergistic_count, additive_count = count_responses(overall_synergy_responses)

    # Determine final result based on combined analysis
    if any_significant_synergy and not any_antagonism:
        overall_result = "Synergistic"
    elif synergistic_count >= additive_count and not any_antagonism:
        overall_result = "Additive"
    elif synergistic_count < additive_count and not any_antagonism:
        overall_result = "Additive"
    elif any_antagonism:
        overall_result = "Antagonistic"
    else:
        overall_result = "Additive"

    return overall_result, reasoning


def save_overall_responses_to_csv(overall_responses, output_csv_path):
    """Save the overall synergy responses of drug combinations to a CSV file."""
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Drugs", "Prediction", "Reasoning"])
        for combination, response in overall_responses.items():
            writer.writerow([', '.join(map(str, combination)), response[0], response[1]])

def read_drug_combinations(input_file_path):
    """Read drug combinations from a CSV file."""
    combinations = []
    with open(input_file_path, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Convert the string of drug IDs into a list of integers
            drug_ids = [int(id.strip()) for id in row['Drug Combination'].split(',')]
            combinations.append(drug_ids)
    return combinations


In [8]:
# Process
input_file_path = "valid_drug_combinations.csv"  # Adjust this to your actual CSV file path
output_csv_path = "model_inputs.csv"

combinations = read_drug_combinations(input_file_path)
print(combinations)
overall_responses = {}

[[568, 92], [694, 92], [910, 92], [274, 272, 41], [1004, 92], [198, 254, 257], [622, 92], [238, 168, 257], [1410, 92], [309, 341, 168], [316, 311, 345, 310], [315, 312], [354, 321], [35, 36, 76, 67, 32], [384, 92], [1020, 92], [1487, 92], [505, 92], [73, 36], [1095, 92], [169, 309, 355, 246], [269, 37, 276, 264, 8], [1674, 92], [528, 92], [320, 26, 321, 362, 337], [1257, 92], [1747, 92], [561, 92], [22, 34], [550, 92], [761, 92], [1126, 92], [231, 76], [95, 286, 17, 168, 77], [204, 318, 330, 356, 317], [868, 92], [1487, 92], [597, 92], [170, 168, 180, 176, 76], [361, 11], [1566, 92], [1637, 92], [1731, 92], [1594, 92], [1909, 92], [257, 221], [710, 92], [1787, 92], [239, 26, 355, 344, 119], [1899, 92], [745, 92], [232, 76, 34, 257, 254], [484, 92], [693, 92], [183, 172, 119, 176, 162], [242, 254], [1979, 92], [1960, 92], [1065, 92], [1520, 92], [1791, 92], [672, 92], [1847, 92], [382, 92], [248, 11, 46, 33, 269], [1604, 92], [60, 51, 41, 67, 252], [639, 92], [871, 92], [1025, 92], [192

In [42]:
for combination in combinations:
    pairwise_synergies = fetch_synergies_from_file(combination)
    prediction, reasoning = analyze_combination_synergy(pairwise_synergies)
    overall_responses[tuple(combination)] = [prediction,reasoning]

save_overall_responses_to_csv(overall_responses, output_csv_path)

#Optionally, load and print the DataFrame for review
df = pd.read_csv(output_csv_path)
print(df.head())

          Drugs Prediction                                          Reasoning
0       568, 92   Additive            BML-190 and Temozolomide are additive. 
1       694, 92   Additive  Carbamazepine and Temozolomide are antagonistic. 
2       910, 92   Additive       Enoxacin and Temozolomide are antagonistic. 
3  274, 272, 41   Additive  Halofantrine and Ethinyl estradiol are additiv...
4      1004, 92   Additive   Foscarnet sodium and Temozolomide are additive. 
